In [ ]:
%tensorflow_version 2.x  # this line is not required unless you are in a notebook
from keras.preprocessing import sequence
import keras
import tensorflow as tf
import os
import numpy as np

In [ ]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

In [ ]:
#Loading from your computer

from google.colab import files
path_to_file = list(files.upload().keys())[0]

In [3]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text)))

Length of text: 1115394 characters


In [4]:
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [17]:
vocab = sorted(set(text))
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)
def text_to_int(text):
  return np.array([char2idx[c] for c in text])

text_as_int = text_to_int(text)

In [18]:
def int_to_text(ints):
  try:
    ints = ints.numpy()
  except:
    pass
  return ''.join(idx2char[ints])

print(int_to_text(text_as_int[:13]))

First Citizen


In [6]:
# lets look at how part of our text is encoded
print("Text:", text[:13])
print("Encoded:", text_to_int(text[:13]))

Text: First Citizen
Encoded: [18 47 56 57 58  1 15 47 58 47 64 43 52]


In [7]:
seq_length = 100  # length of sequence for a training example
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

In [9]:
def split_input_target(chunk):  # for the example: hello
    input_text = chunk[:-1]  # hell
    target_text = chunk[1:]  # ello
    return input_text, target_text  # hell, ello

dataset = sequences.map(split_input_target)  # we use map to apply the above function to every entry

In [11]:
BATCH_SIZE = 64
VOCAB_SIZE = len(vocab)  # vocab is number of unique characters
EMBEDDING_DIM = 256
RNN_UNITS = 1024

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

data = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

In [12]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.LSTM(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

model = build_model(VOCAB_SIZE,EMBEDDING_DIM, RNN_UNITS, BATCH_SIZE)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           16640     
_________________________________________________________________
lstm (LSTM)                  (64, None, 1024)          5246976   
_________________________________________________________________
dense (Dense)                (64, None, 65)            66625     
Total params: 5,330,241
Trainable params: 5,330,241
Non-trainable params: 0
_________________________________________________________________


In [13]:
for input_example_batch, target_example_batch in data.take(1):
  example_batch_predictions = model(input_example_batch)  # ask our model for a prediction on our first batch of training data (64 entries)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")  # print out the output shape

(64, 100, 65) # (batch_size, sequence_length, vocab_size)


In [14]:
# we can see that the predicition is an array of 64 arrays, one for each entry in the batch
print(len(example_batch_predictions))
print(example_batch_predictions)

64
tf.Tensor(
[[[ 0.00181528 -0.0032021  -0.00162083 ... -0.00111642 -0.00113384
    0.00210502]
  [-0.00049912 -0.00338957  0.00084892 ... -0.00297879 -0.00195744
    0.00699059]
  [ 0.00436914 -0.00040307 -0.0020912  ...  0.0010442   0.00295802
    0.01066749]
  ...
  [-0.0023037  -0.01538264 -0.0123384  ...  0.00449     0.0046932
   -0.00575491]
  [-0.00491805 -0.01452674 -0.00651431 ...  0.00034614  0.00245755
    0.00296883]
  [-0.00872178 -0.01095952 -0.00085584 ...  0.00178854 -0.00010347
    0.00782594]]

 [[-0.00309516 -0.00153147  0.00200987 ... -0.00151538 -0.00148843
    0.00688836]
  [-0.00262828  0.00069556  0.00527932 ...  0.0012317   0.00224588
   -0.00338847]
  [ 0.00201553  0.00257312  0.00056883 ...  0.00553857  0.00575582
    0.00471671]
  ...
  [ 0.00631589 -0.0025674   0.00236145 ...  0.00166858 -0.00056066
    0.00453069]
  [ 0.00571607 -0.00566844 -0.00270624 ... -0.00055656  0.00585685
    0.00523234]
  [ 0.00693676 -0.0158962  -0.00470471 ...  0.00210318  0.00

In [15]:
# lets examine one prediction
pred = example_batch_predictions[0]
print(len(pred))
print(pred)
# notice this is a 2d array of length 100, where each interior array is the prediction for the next character at each time step

100
tf.Tensor(
[[ 0.00181528 -0.0032021  -0.00162083 ... -0.00111642 -0.00113384
   0.00210502]
 [-0.00049912 -0.00338957  0.00084892 ... -0.00297879 -0.00195744
   0.00699059]
 [ 0.00436914 -0.00040307 -0.0020912  ...  0.0010442   0.00295802
   0.01066749]
 ...
 [-0.0023037  -0.01538264 -0.0123384  ...  0.00449     0.0046932
  -0.00575491]
 [-0.00491805 -0.01452674 -0.00651431 ...  0.00034614  0.00245755
   0.00296883]
 [-0.00872178 -0.01095952 -0.00085584 ...  0.00178854 -0.00010347
   0.00782594]], shape=(100, 65), dtype=float32)


In [16]:
# and finally well look at a prediction at the first timestep
time_pred = pred[0]
print(len(time_pred))
print(time_pred)
# and of course its 65 values representing the probabillity of each character occuring next

65
tf.Tensor(
[ 1.8152810e-03 -3.2021047e-03 -1.6208319e-03 -5.9873017e-04
 -8.5134963e-03 -2.7945682e-03  1.2221680e-03  1.3203216e-03
 -1.3463397e-03 -9.7077386e-04 -2.0272806e-03 -5.1591657e-03
  5.9295059e-03 -3.6178287e-03  3.8232439e-04  7.6406728e-04
  6.0175708e-04  6.4078434e-03 -4.6254541e-03  2.2368340e-03
  2.0445243e-04  2.1967907e-03 -5.7662334e-03  1.6075552e-03
 -1.6739077e-03 -3.9620561e-04 -1.0825646e-03 -1.5434134e-03
  5.5689197e-03  3.9283759e-03  2.0989762e-03 -4.1235648e-03
 -9.9919166e-04 -1.5845560e-03 -2.1739593e-03  3.9905338e-03
  1.8964219e-03  3.2458045e-03 -6.3445945e-03 -1.0060651e-03
  7.0616277e-04 -2.3638888e-03 -1.9484926e-03  8.5225794e-05
  3.4825373e-03 -2.0936891e-03 -2.2039786e-03 -8.1065006e-04
 -2.5797694e-03  1.2380087e-03 -4.0932512e-03  1.7486496e-03
  2.3796062e-03  2.0173807e-03 -1.8791487e-03  4.1361083e-03
 -2.5960631e-03  4.0450813e-03  6.6477805e-03  7.4479496e-04
 -5.3987554e-03  2.9054312e-03 -1.1164208e-03 -1.1338411e-03
  2.105019

In [19]:
# If we want to determine the predicted character we need to sample the output distribution (pick a value based on probabillity)
sampled_indices = tf.random.categorical(pred, num_samples=1)

# now we can reshape that array and convert all the integers to numbers to see the actual characters
sampled_indices = np.reshape(sampled_indices, (1, -1))[0]
predicted_chars = int_to_text(sampled_indices)

predicted_chars  # and this is what the model predicted for training sequence 1

"\ntBCXOdEZgim\npjeF-HI;YGNbux\nYU!\nNFucu,eupY&COCzU;3sEswSFnXNrpEcCu$Us?-zwGmc&Suvr$\n:bFTEA-t-&.?y'DQGT"

In [20]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [21]:
model.compile(optimizer='adam', loss=loss)

In [22]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [23]:
history = model.fit(data, epochs=100, callbacks=[checkpoint_callback])

Epoch 1/100
172/172 [==============================] - 10s 56ms/step - loss: 2.6063
Epoch 2/100
172/172 [==============================] - 10s 56ms/step - loss: 1.9030
Epoch 3/100
172/172 [==============================] - 10s 56ms/step - loss: 1.6499
Epoch 4/100
172/172 [==============================] - 10s 55ms/step - loss: 1.5122
Epoch 5/100
172/172 [==============================] - 10s 56ms/step - loss: 1.4294
Epoch 6/100
172/172 [==============================] - 10s 55ms/step - loss: 1.3711
Epoch 7/100
172/172 [==============================] - 10s 55ms/step - loss: 1.3259
Epoch 8/100
172/172 [==============================] - 10s 55ms/step - loss: 1.2864
Epoch 9/100
172/172 [==============================] - 10s 56ms/step - loss: 1.2509
Epoch 10/100
172/172 [==============================] - 10s 55ms/step - loss: 1.2164
Epoch 11/100
172/172 [==============================] - 10s 55ms/step - loss: 1.1821
Epoch 12/100
172/172 [==============================] - 10s 56ms/step - lo

In [24]:
model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, batch_size=1)

In [25]:
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

In [ ]:
checkpoint_num = 10
model.load_weights(tf.train.load_checkpoint("./training_checkpoints/ckpt_" + str(checkpoint_num)))
model.build(tf.TensorShape([1, None]))

In [27]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 800

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
    
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the character returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted character as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [28]:
inp = input("Type a starting string: ")
print(generate_text(model, inp))

Type a starting string: citizen
citizens and friends to Rome,
And die no more. Gond grants that fear the court?

CAMILLO:
My liege,
Your honour never followed.

ROMEO:
So long courter for a hair of my strander in his doom.

Shepherd:
This but stop never prayer. A friar tale some peace
With smoter'd lady and deserved thee!

Purse:
They say 'There's none so. Camillo, and I their turndsmen.

Second Senator:
Our army is stopp'd with our company.

KING HENRY VI:
Where did you prove up, I think, in a vault very like,
The horror else?'

SICINIUS:
You shall go beth:
A good confirmer with you; yet two no reason too, for I am proof against that title and his fellows,
I'll prove the county of his post to faith in fearful consul
With quarrel silken dial, art's hand.

LADY ANNE:
In thy foul throat of war behavior
From the waster of men of m
